# T1049 - System Network Connections Discovery
Adversaries may attempt to get a listing of network connections to or from the compromised system they are currently accessing or from remote systems by querying for information over the network. 

An adversary who gains access to a system that is part of a cloud-based environment may map out Virtual Private Clouds or Virtual Networks in order to determine what systems and services are connected. The actions performed are likely the same types of discovery techniques depending on the operating system, but the resulting information may include details about the networked cloud environment relevant to the adversary's goals. Cloud providers may have different ways in which their virtual networks operate.(Citation: Amazon AWS VPC Guide)(Citation: Microsoft Azure Virtual Network Overview)(Citation: Google VPC Overview) Similarly, adversaries who gain access to network devices may also perform similar discovery activities to gather information about connected systems and services.

Utilities and commands that acquire this information include [netstat](https://attack.mitre.org/software/S0104), "net use," and "net session" with [Net](https://attack.mitre.org/software/S0039). In Mac and Linux, [netstat](https://attack.mitre.org/software/S0104) and <code>lsof</code> can be used to list current connections. <code>who -a</code> and <code>w</code> can be used to show which users are currently logged in, similar to "net session". Additionally, built-in features native to network devices and [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) may be used (e.g. <code>show ip sockets</code>, <code>show tcp brief</code>).(Citation: US-CERT-TA18-106A)

## Atomic Tests

### Atomic Test #1 - System Network Connections DiscoveryGet a listing of network connections.

Upon successful execution, cmd.exe will execute `netstat`, `net use` and `net sessions`. Results will output via stdout.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
netstat
net use
net sessions
```

In [ ]:
Invoke-AtomicTest T1049 -TestNumbers 1

### Atomic Test #2 - System Network Connections Discovery with PowerShellGet a listing of network connections.

Upon successful execution, powershell.exe will execute `get-NetTCPConnection`. Results will output via stdout.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Get-NetTCPConnection
```

In [ ]:
Invoke-AtomicTest T1049 -TestNumbers 2

### Atomic Test #3 - System Network Connections Discovery FreeBSD, Linux & MacOS
Get a listing of network connections.

Upon successful execution, sh will execute `netstat` and `who -a`. Results will output via stdout.

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `sh`!
##### Description: Check if netstat command exists on the machine

##### Check Prereq Commands:
```sh
if [ -x "$(command -v netstat)" ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
echo "Install netstat on the machine."; exit 1;

```

In [ ]:
Invoke-AtomicTest T1049 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
netstat
who -a
```

In [ ]:
Invoke-AtomicTest T1049 -TestNumbers 3

### Atomic Test #4 - System Discovery using SharpView
Get a listing of network connections, domains, domain users, and etc.  
sharpview.exe located in the bin folder, an opensource red-team tool.
Upon successful execution, cmd.exe will execute sharpview.exe <method>. Results will output via stdout.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sharpview.exe must exist on disk at specified location (#{SharpView})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\SharpView.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\..\ExternalPayloads\SharpView.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest https://github.com/tevora-threat/SharpView/blob/b60456286b41bb055ee7bc2a14d645410cca9b74/Compiled/SharpView.exe?raw=true -OutFile "PathToAtomicsFolder\..\ExternalPayloads\SharpView.exe"

```

In [ ]:
Invoke-AtomicTest T1049 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$syntaxList = "Invoke-ACLScanner", "Invoke-Kerberoast", "Find-DomainShare" 

foreach ($syntax in $syntaxList) {
PathToAtomicsFolder\..\ExternalPayloads\SharpView.exe $syntax -}
```

In [ ]:
Invoke-AtomicTest T1049 -TestNumbers 4

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as Lateral Movement, based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Further, [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) commands may also be used to gather system and network information with built-in features native to the network device platform.  Monitor CLI activity for unexpected or unauthorized use commands being run by non-standard users from non-standard locations. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).


## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
A defender can manipulate the output of commands commonly used to enumerate a system's network connections. They could seed this output with decoy systems and/or networks or remove legitimate systems from the output in order to direct an adversary away from legitimate systems.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.
